<a href="https://colab.research.google.com/github/sourabhnayak04444/Image-Resizer-Tool/blob/main/Image_resizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Pillow (PIL)
!pip install Pillow

In [ ]:
import os
from PIL import Image
from google.colab import files # Essential for local file operations in Colab
import shutil

def resize_images_batch(input_folder, output_folder, target_width):
    """
    Resizes all supported images in a folder to a target width while
    maintaining the aspect ratio, and saves them as JPEG files.
    """
    # 1. Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"Created output directory: {output_folder}")

    supported_formats = ('.jpg', '.jpeg', '.png')
    processed_count = 0

    # 2. Loop through all files in the input folder
    for filename in os.listdir(input_folder):
        if filename.lower().endswith(supported_formats):
            try:
                input_path = os.path.join(input_folder, filename)

                # 3. Open and Resize the image
                img = Image.open(input_path)
                original_width, original_height = img.size

                # Only resize if the image is wider than the target width
                if original_width > target_width:
                    ratio = target_width / original_width
                    target_height = int(original_height * ratio)
                    resized_img = img.resize((target_width, target_height), Image.Resampling.LANCZOS)
                else:
                    # If the image is smaller, just use the original
                    resized_img = img
                    print(f"Skipped resizing {filename}: already smaller than {target_width}px.")


                # 4. Save the resized image as JPEG
                name, ext = os.path.splitext(filename)
                output_filename = f"{name}_resized.jpeg"
                output_path = os.path.join(output_folder, output_filename)

                resized_img.save(output_path, 'jpeg', quality=90)

                print(f"Successfully processed and saved: {output_filename}")
                processed_count += 1

            except Exception as e:
                print(f"Error processing {filename}: {e}")

    print(f"\n--- Batch Resize Complete ---")
    print(f"Total images processed: {processed_count}")

    # Clean up the input folder after processing
    shutil.rmtree(input_folder)

# --- Configuration using Colab's local /content directory ---
INPUT_DIR = 'input_images'  # Will be /content/input_images
OUTPUT_DIR = 'output_images' # Will be /content/output_images
NEW_WIDTH = 800
# ---------------------

# The execution block is now modified to handle upload/download
if __name__ == '__main__':
    # Ensure the input folder exists temporarily
    if not os.path.exists(INPUT_DIR):
        os.makedirs(INPUT_DIR)

    print("Step 1:Please select the image files from your PC to upload.")
    # This opens a file selection dialogue on your PC
    uploaded = files.upload()

    # Move uploaded files from /content/ to the input folder
    for name, data in uploaded.items():
        with open(os.path.join(INPUT_DIR, name), 'wb') as f:
            f.write(data)
        print(f"File uploaded: {name}")

    # Run the resizing process
    print("\nStep 2:Running image batch resize...")
    resize_images_batch(INPUT_DIR, OUTPUT_DIR, NEW_WIDTH)

    # Download the resized files back to your PC
    print("\nStep 3:Zipping and downloading the results.")

    # 1. Zip the output folder
    # shutil.make_archive(base_name, format, root_dir, base_dir)
    zip_filename = 'resized_images_output'
    shutil.make_archive(zip_filename, 'zip', OUTPUT_DIR)

    # 2. Download the zip file to your local machine
    files.download(f'{zip_filename}.zip')